<span type="title">Spring IOC</span> | <span type="update">2018-09-30</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">这一部分主要介绍使用 Spring 框架进行 Web 开发简化的方法。本章重点介绍了 Spring 的 IOC/DI 控制反转/依赖注入，包括使用 XML 和 注解两种以无侵入性的方式创建对象实例的方法。IOC 是工厂方法的升级版本，其优点在于：使用 XML，依赖 Spring 提供的IOC容器，对象可以更加方便的在运行时动态生成，而不需手动构造（可以将 Spring IOC 看作一个大的对象工厂，其根据 XML 进行 RTTI 参数注入，从容器中根据 id 获取对象。）。</p></span>

# Spring IOC 概要

## IOC 使用方法一览

Spring 的 IOC 需要使用以下 jar 依赖：commons-logging, core, beans, context, expression.

其基本使用方法如下：

person Bean 类

```java
public class Person {
    private String name;
    private int id;
    private Address address;
    private List<Address> schools;
... 略}
```

IOC 容器 Bean 配置文件

```xml
<bean id="ccnu" class="com.mazhangjing.test.Address" scope="singleton">
    <constructor-arg index="0" value="China"/>
    <constructor-arg name="city" value="Wuhan" />
    <constructor-arg name="code" value="027" />
    <constructor-arg index="3" value="CCNU" />
</bean>
<bean id="person" class="com.mazhangjing.test.Person" scope="singleton">
    <property name="name" value="Corkine"/>
    <property name="address" ref="ccnu" />
    <property name="id">
        <value>2017110658</value>
    </property>
</bean>
```

IOC 对象调用

```java
ApplicationContext ctx = new ClassPathXmlApplicationContext("applicationContext.xml");
Person person = (Person) ctx.getBean("person");
```

通过在 context.xml 进行配置，然后 IOC 容器会自动解析此 XML 文件，并且进行 Bean 的构造，并且自动管理其生命周期。当需要使用 Bean 的时候，从 ApplicationContext 中根据 id 获取对象即可使用。这就是 IOC 容器的基本使用方法。重点在于 XML 配置 Bean 以及其依赖关系、各种数据结构、外部属性文件的语法。

## IOC/DI 的定义

IOC 指的是 Inversion of Control。在传统的 JavaEE 中，当希望获取一个对象，我们要向容器发送请求，获取构造此对象的资源，容器进行相应，然后构造此对象。在IOC中，容器主动的在项目启动的时候，自动将合适的资源注入对象中，这样，只需要向容器寻求这个对象即可，而不用再去调用和配置对象构造器来手动管理对象构建。IOC和DI含义相同，其中DI指的是依赖注入，也就是容器将对象的类属性所需要的对象使用 setter 方法进行自动调用，然后既可以直接使用这个对象，而不需手动的去 set 依赖的方式。DI是IOC的另一种表述形式。

## IOC/DI 的前世今生

IOC 产生的前世今生是这样的：我们现在有很多对象，这些对象需要被协调用来做某些事情，为了对象和对象间的互相通信，我们一般会在一个对象内保留其余其可能与之交互对象的实例，实例通过构造器传入或者通过 setter 方法进行设置，进行设置的过程称之为对象的装配。

不难发现，对象的装配是一件复杂的事情，因为，我们在进行某项工作的时候，为了得到与之交互的对象，不能总是就地新建一堆依赖的对象实例，然后进行对象的装配和使用。为了做到对象和对象的兼容以及协调，我们必须在对象内保存其余对象的实例引用，因此就必须有一个中央提供其所需的依赖组件，为其提供实例引用。其次，如果我们真的就地创建实例引用，那么这些创建此对象的代码将难以被其余需要相同依赖的其它对象进行重用。因此，我们必须在某个地方集中创建类实例，然后分配这些实例引用给对象进行装配。第一次革命是解决这个问题而产生的，革命的结果是：抽象工厂。

抽象工厂显著的降低了对象装配的负担。对象只需要从上下文获取工厂的一个引用即可获取工厂提供的对象实例引用，从而用来进行对象组装。对象甚至不需要知道工厂实际初始化的是什么类型的对象，以及为其接口提供服务的对象是从哪里来的。尽管拥有这些好处，但是抽象工厂亦有其缺点。抽象工厂的问题在于，其接口限制了从其中调用对象类型的灵活性。

IOC/DI就是为了解决此问题而生的：在DI中，容器充当了抽象工厂，调用工厂通过 id 获得实例引用即可。容器在创建时会自动的为缺少依赖的对象注入依赖所需要的“单例”对象引用，这样对象就可以直接拿来使用了。

## IOC/DI 的本质

IOC 本质上还是一个 工厂，对象通过统一接口 `ApplicationContext.getBeans()`，从 IOC 获取实例。不过这个工厂通过 RTTI 的方式解决了获取对象的接口僵硬并且难以改变的问题。这个工厂并非是一个 Java 类，其被 Spring 自动管理，为了使用 RTTI，设置工厂需要传入参数以说明应该构造哪些 Beans，通常通过 Bean Config XML 或者 Configuration 文件来提供配置。

# IOC 的装配：自动扫描

Spring 4 以后，推荐使用自动化装配方案，这种方案避免了 XML 设置的冗长，同时对于 Java 也没有过多的侵入性（只是添加了一些注解而已，并且这些注解很多还是JavaEE兼容的）。自动化装配技术关键有二：component scanning 组件扫描以及 autowiring 自动装配。组件扫描是为了将依赖搜集整理，然后在容器中进行 Bean 的构造， autowiring 自动装配则是自动将这些依赖进行对象的注入。

首先创建一个接口 CompactDisc：

```java
public interface CompactDisc { void play(); }
```

接着提供两个实现：

```java
@Component(value = "disc")
public class SgtPeppers implements CompactDisc {
    protected String title = "The Night";
    protected String artist = "Jay Chou";
    @Override
    public void play() {
        out.println(title + " " + artist + " is playing...");
    }
}
@Component
public class Sgt2Peppers extends SgtPeppers {
    @Override
    public void play() {
        out.println(title + " " + artist + " is playing 222 ...");
    }
}
```

接着又是一个组件，这个组件整合了上面的组件，不过没有注入，甚至没有提供 setter 方法！！！

```java
@Component(value = "test")
public class Test {

    @Autowired(required = false)
    public CompactDisc disc;

    public CompactDisc ndisc;

    @Autowired
    public void setNdisc(CompactDisc sgt2Peppers) {
        this.ndisc = sgt2Peppers;
    }
}
```

配置文件如下：

```java
@Configuration
@ComponentScan(
        basePackageClasses = {CompactDisc.class,Test.class},
        basePackages = "com.mazhangjing.sound")
public class CDPlayerConfig { }
```

测试代码如下：

```java
public static void main(String[] args) {
    ApplicationContext ctx = new AnnotationConfigApplicationContext(CDPlayerConfig.class);
    Test test = (Test) ctx.getBean("test");
    test.disc.play();
    test.ndisc.play();
}
```



对于自动装配，可以看到，关键部分在于，对于被依赖的组件设置 @Component 注解/ @Named 注解，然后添加配置文件，设置 @Configuration 以及 @ComponentScan 选取扫描并且自动装备组件 Bean 的范围。之后在我们的代码中从 ApplicationContext 获取组件 Bean 即可。

# IOC 的装配：Java 代码

 在任何情况下，都应该优先使用自动装配，但是，自动装配存在这样的问题，如果你不想重新编译那些写好的代码，或者你不能重新编译，那么，我们就必须通过显式的声明来指定组件Beans的信息。自动装配更强调一种“约定”，比如id默认为方法名开头小写，JavaWeb中经常使用单例等等。Java代码装配看起来就和实际实现一个“抽象工厂”类似，不过不用实现什么特定的接口。
 
Java 代码如下，我们为 SgtPeppers 添加了一个构造器，添加了一个 Song 类型以演示代码装配的复杂情况。

```java
SgtPeppers(String title,String artist) {
    this.title = title;
    this.artist = artist;
}
public class Song {
    String name;
    String title;
}
```

```java
@Configuration
public class CDPlayerConfig {
    @Bean CompactDisc sgtA() {
        return new SgtPeppers();
    }
    @Bean(name = "sgtC")
    CompactDisc sgtB() {
        return new Sgt2Peppers();
    }
    @Bean Song song() {
        Song song = new Song();
        song.title = "叶惠美";
        song.name = "Jay Chou";
        return song;
    }
    @Bean CompactDisc sgtD(Song song) {
        print(song());
        return new SgtPeppers(song.title,song.name);
    }
    @Bean CompactDisc sgtE() {
        print(song());
        return new SgtPeppers(song().title,song().name);
    }
}
```

**没有构造器的简单Bean创建**

如果不自动扫描，那么对于 Configuration 文件是不用添加 ComponentScan 注释的。在 Configuration 类中，每一个注释了 @Bean 的方法都必须返回一个对象，如果不指定 Bean 的 name 参数，那么默认创建的 Bean 的 id 为方法名，比如 sgtA，sgtC，sgtB 方法因为主动指定了 name，因此使用 sgtC 作为生成Bean的 id。

**含有构造器的复杂Bean的Bean调用截获**

如果我们创建的对象有构造器，比如 sgtD 方法，这里的 song 参数将会从 Bean 中 id = “song” 的那个 Bean 中获取，也就是 song() 方法创建的那个 Bean。注意 sgtE 方法，当我们调用 song() 方法时，Spring 会拦截对于这个 Bean 注解的方法的调用，然后返回 Bean 实例，而不是允许我们多次调用 song() 方法创建多个 song 实例。通过打印 song() 看到，Hash 值标明 Spring 确实进行了拦截，并且返回的是同样实例的引用。`com.mazhangjing.sound.Song@1ee807c6`

这里的 sgtD 中的 song 以及 sgtE 中的 song() 指的都是 id = "song" 的那个 Bean。注意，这里并不一定要主动创建一个 song 的 Bean，可以通过 autowired（`@Component class Song {}` or `@Component(value="song") class XXX {}`） 或者 XML（`<bean class="xxx" id="song">`） 方式扫描或者创建这样一个对象，只要这个 id 的 Bean 在 Spring IOC 的容器作用域内即可。

# IOC 的装配 : 通过 XML

XML 装配是 Spring 最为传统的一种方式，当然，也较为繁琐。XML 必须声明正确的命名空间，同时添加适当的前缀，必须以 Beans 标签作为根部元素。

```xml
<?xml version="1.0" encoding="UTF-8"?>
<beans xmlns="http://www.springframework.org/schema/beans"
       xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
       xmlns:p="http://www.springframework.org/schema/p"
       xmlns:util="http://www.springframework.org/schema/util"
       xsi:schemaLocation="http://www.springframework.org/schema/beans
       http://www.springframework.org/schema/beans/spring-beans.xsd
       http://www.springframework.org/schema/util
       http://www.springframework.org/schema/util/spring-util-4.0.xsd">
</beans>
```

一个最基本的 Bean 应该指定两个字段 id 和 class，这样会调用对应 class 的类的无参构造器进行元素构造，存放在 Spring IOC 容器指定 id 的引用名下。

```xml
<bean id="ccnu" class="com.mazhangjing.test.Address"/>
```

## 通过构造器标签进行 Bean 构造

```xml
<bean id="ccnu" class="com.mazhangjing.test.Address" scope="singleton">
    <constructor-arg index="0" value="China"/>
    <constructor-arg name="city" value="Wuhan" />
    <constructor-arg name="code" value="027" />
    <constructor-arg index="3" value="CCNU" />
</bean>
```

如上所示，constructor-arg 标签的 value 值标明传入的参数，可以混合使用 index 和 name 来指定参数对应的顺序。这种方式不需要用无参构造器。此外，传入的value如果是字符串字面量，可以自动转换为 String 以及基本类型和其包装类型，但是对于别的类型，需要使用 ref 字段来引用别的 Bean 作为参数，或者直接在内部写一个内部的 Bean。（和Java代码装配的参数传入类似）

此外，你可以使用 c 命名空间来传入构造参数，可选顺序和名称两种方式：

`xmlns:c="http://www.springframework.org/schema/c"`
```xml
<bean id="ccnu2" class="com.mazhangjing.test.Address" c:city="Wuhan" c:code="027" c:country="China" c:home="CCNU" />
<bean id="ccnu2" class="com.mazhangjing.test.Address" c:_0="China" c:_1="Wuhan" c:_2="027" c:_3="CCNU" />
```

## 通过setter标签进行 Bean 构造

```xml
<bean id="person2" class="com.mazhangjing.test.Person" scope="singleton">
    <property name="name" value="Marvin"/>
    <property name="address" ref="ccnu" />
    <property name="id">
        <value>2017110659</value>
    </property>
    <property name="schools" ref="mylist"/>
</bean>
```

注意，可以使用 setter 进行依赖注入，使用 property 属性即可，name 表示 setter 的属性名称，value 表示其值。其中 value 的值可以用体，或者 value 属性。这里展示了 ref 的使用方法，其中 ccnu 指的是 一个 id = ccnu 的 Bean。这里如果不希望创建一个 Bean 来做 ref，可以直接在 property 的体中嵌套一个 Bean，作为内部 Bean，这个内部 Bean 只用于构造此 property，不能在外部引用（相当于局部变量）。

我们可以采用 p 命名空间来简化配置，如下所示，在一个 标签内直接用 p 标签来调用原本在主命名空间内的标签，可以直接用来对Bean的属性进行设置或者引用，看起来更加方便。

```xml
<bean id="person3" class="com.mazhangjing.test.Person" p:address-ref="ccnu" p:id="2017110658" p:schools-ref="mylist" p:name="Lucy"/>
```

此外需要注意，对于 c 和 p 而言，设置 `c:_0-ref` `p_prop-ref` 后面的 ref 表示引用，必须是一个 bean 的 id。


## 基本数据结构的 Bean

如果参数需要基本的一些数据结构，比如 map、list、set 等，可以直接用标签即可：

```xml
<bean id="teacher" class="com.mazhangjing.test.Teacher">
    <property name="addressMap">
        <map>
            <entry key="before" value-ref="hzau" />
            <entry key="now" value-ref="hzau" />
        </map>
    </property>
    <property name="schools">
    <list value-type="com.mazhangjing.test.Address">
        <ref bean="hzau"/>
        <ref bean="ccnu"/>
    </list>
    </property>
</bean>
```

注意上面的 property 的值是一个 map 对象，map 对象必须有 entry 对象组成，填入 entry 的 key 和 value-ref（或者 value） 属性以填充 map。下面的 property 是一个 list，这里并没有直接填充内容，而是使用了 ref指向外部 Bean 作为 list 的内容。set 同理。

注意，这些 map、list、set 同样作为局部变量，如果我们希望放置在外部以方便别的对象引用（作为 Bean），需要引入 util 命名空间。

```xml
<util:list id="mylist">
    <ref bean="hzau" />
    <ref bean="ccnu" />
</util:list>
```

## Bean 的作用域

Bean 默认的 scope 为 singleton，这意味着单例，选择 prototype 表示每次调用均创建一个实例。Bean 可以用两种方式进行参数传递和构造，分别是通过构造器、setter、静态工厂、实例工厂的方法，其中使用较多的构造器和setter方法。


## Bean 的转义和空值

对于 null 值，采用 `<null/>` 标签，虽然如果你不写，默认为 null。

对于需要转义的XML标记，采用 `<![CDATA[ "fuck_strings" ]]>` 进行转义。

## Bean 的自动装配

对于 XML 声明的方式，也可以用来进行自动装配。需要指定按类型还是按名称进行装配，使用 autowire 字段，常用 byName，因为可能有一个接口的多个 Bean，byType 会造成混乱。此 bean 的所有属性，都会自动的 Spring 寻找对应 id 的对应 type 的 Bean 进行填充。

```xml
<bean id="robot" class="com.mazhangjing.test.Person" p:name="Marvin-Robot" autowire="byName" />
```

此外，可以通过构造器进行装配，不过其过于复杂。使用XML自动装配的问题在于，由于没有注解，因此默认 autowire 对于所有的类实例都进行了设置，此外，byName 要求 id 必须和 bean 的属性名称一致，在实际使用中容易造成混淆。因此，不推荐使用 XML 自动装配，如果有自动装配需求，使用注解进行。

## Bean 的继承和依赖

此处的继承不是 OOP 的继承，这里的继承的意思是，子 Bean 默认使用父 Bean 的配置，如果不特殊指定的话。父 Bean 可以设置为 abstract，这样就不可构造为对象，只提供配置模板，但是并非所有父 Bean 属性都会被继承，autowire 和 abstract 都不会被继承。父 Bean 和 子 Bean 的类可能不同（但是此时必须设置父Bean为 abstract）。

依赖指的是 depend-on 属性，指定依赖后，会先构造依赖的 Bean 对象，然后才是当前的 Bean。

```xml
<bean id="father-hzau" class="com.mazhangjing.test.Address" abstract="true">
    <constructor-arg index="0" value="China"/>
    <constructor-arg name="city" value="Wuhan" />
    <constructor-arg name="code" value="027" />
    <constructor-arg index="3" value="HZAU" />
</bean>

<bean id="hzau" class="com.mazhangjing.test.Address" scope="singleton" parent="father-hzau" depends-on="person">
    <property name="home" value="real hzau" />
</bean>

<bean id="hzau-sun" class="com.mazhangjing.test.Address" parent="hzau">
    <property name="home" value="Some Where of hazu" />
</bean>
```

## 通过静态工厂方法创建 Bean

工厂方法使用很少。通过静态工厂，意思是，这个工厂类需要提供一个类、一个用于获取 Bean 的静态方法，这个方法应该接受一个字符串查询，返回一个 Bean 对象。通过 XML 中的 class 设置工厂，factory-method 指定此方法，通过 constractor-arg 传递参数即可创建此对象为 Bean。

一般而言，静态工厂提供一个静态容器，一个静态方法通过传入参数获取静态容器的值。其XML配置如下：

```xml
<bean id="obj1" class="com.mazhangjing.test.StaticFactory" factory-method="getObject" c:_0="obj1"/>
```

注意，这里的 constractor 是用作方法的参数传入，此外，此处的 id 是创建的对象的 id，而不是工厂的 id。

## 通过实例工厂方法创建 Bean

实例工厂和静态工厂的区别在于，后者的方法并非静态的，此外，后者需要单独创建一个 Beans，然后在对象的 bean 中指定实例工厂的方法，是用 `factory-bean` 字段指定。此外的 `factory-method` 和 `c:_0` 和 静态工厂类似。此外，实例工厂的 bean 创建的对象不需要写 class 即可。

```xml
<bean id="factory" class="com.mazhangjing.test.Factory" />
<bean id="obj2" factory-bean="factory" factory-method="getObject" c:_0="obj2" />
```

## 通过 FactoryBean 创建 Bean

此种方式和静态工厂类似，bean 直接使用 FactoryBean 的类，此类应该实现 getObject 和 getObjectType，返回指定 Bean 的对象。可以通过参数进行设置，然后构造在 getObject 中返回特定的 Bean。

```java
public class DBFactoryBean implements FactoryBean<DB> {
    private String username;

    public void setUsername(String username) { this.username = username; }
                                                       
    public Class<?> getObjectType() {
        return DB.class;
    }

    public DB getObject() {
        DB db = new DB();
        db.setName(username);
        return db;
    }

    public boolean isSingleton() {
        return true;
    }
}
```

```xml
<bean id="db2" class="com.mazhangjing.test.DBFactoryBean">
    <property name="username" value="Corkine Ma" />
</bean>
```

## XML 中的自动装配

自动装配需要指定 Java 类的组件注解，包括 @Component 和 @Respository 以及 

## 外部属性文件链接

XML 文件很容易变得复杂，我们可以使用 context:property-placeholeder 将其链接到外部属性文件中，如下，采用 EL 表达式来获取外部属性文件的字段。注意，需要引入 context 前缀。

`xmlns:context="http://www.springframework.org/schema/context"`


db.properties 文件内容：
```
user=CorkineMa
passwd=22202021
information=Something New
```

注意，使用 classpath 来获取此文件实际路径。在使用前必须先提供 context 的属性占位标签，像 EL 表达式一样使用即可。

```xml
<context:property-placeholder location="classpath:db.properties" />
<bean id="db" class="com.mazhangjing.test.DB" p:name="${user}" p:passwd="${passwd}" p:information="${information}" />
```


## 在 XML 中使用 SpEL

SpEL 是一种类似于 EL 的表达式语言，其之处运行时查询和对象操作，采用 `#{...}` 作为定界符， SpEL 可以用来对于 bean 进行引用，调用方法、引用属性、计算值、正则匹配。

字面量的表示： `#{5},  #{80.7},  #{1e4},  #{'Check'},  #{false}`

引用其它对象：`#{ccnu}` 即 beans 的 id。 `#{ccnu.name}` 即 beans 的 id 对应的 bean 属性。 `#{ccnu.toString()}` 即 beans 的 id 对应的方法，方法支持链式操作 `#{ccnu.name.toString().toUpperCase()}`。 

运算：`#{2 + 2},  #{"hello" + ", " + world},  #{counter.total / counter.count},  #{2 ge 4},  #{2 >= 4}`

静态方法/属性调用： `#{T(java.lang.Math).PI}` 注意，静态方法类型放置在 T() 括号中，然后使用点号调用静态方法。

# IOC 的使用

Spring IOC 容器的实现有二：BeanFactory 以及 ApplicationContext。后者是前者的子接口，面向开发者，在任何情况下都应该使用后者进行Bean的获取。AC的主要实现类有 ClassPathXmlApplicationContext 以及 FileSystemXmlApplicationContext。其中直接继承自 AC 的是 ConfigurableApplicationContext，而 CA 和 FA 都是继承自 CAC 的。CAC 相比于 AC，主要添加了 refresh 和 close 方法，用于启动、刷新和关闭上下文。AC 的初始化上下文的时候就已经实例化所有的单例Bean了。

其使用举例如下：

```java
ApplicationContext ctx = new ClassPathXmlApplicationContext("applicationContext.xml");
Person person = (Person) ctx.getBean("person");
Teacher teacher = ctx.getBean(Teacher.class);
```

注意，可以使用 `Class<T>` 和 String 作为参数，前者的问题在于，一种类型只能在 IOC 中有一个对象，否则会报错，无法取出对象。对于基于注解的 IOC 使用举例如下：

```java
ApplicationContext ctx = new AnnotationConfigApplicationContext(CDPlayerConfig.class);
Test test = (Test) ctx.getBean("test");
test.disc.play();
```

同上，可以在 AC 对象获取时使用 .class 指向特定配置文件，或者使用空构造器，之后使用register来注册配置，使用refresh来刷新上下文。

# IOC 的 Bean 生命周期

Bean 的生命周期如下：IOC 容器通过构造器或者工厂方法创建 Bean 实例，然后为 Bean 的属性设置值和其他 Bean 的引用。调用 Bean 的初始化方法，当容器关闭时，调用 Bean 的销毁方法。

## XML 中的初始化和销毁

通过在 xml 中声明 init-method 和 destory-method 可以在此 Bean 初始化和销毁时调用 Java 方法进行处理。注意，Address.class 中必须提供 init() 和 destory() 方法。

```xml
<bean id="ccnu" class="com.mazhangjing.test.Address" scope="singleton" init-method="init" destroy-method="destory" />
```

## 使用后置处理器进行管理

注意，和 SE 不同，Bean 的 生命周期的 init 是在 Bean 被 setter 和注入后才可用的。我们通过使用 Bean 后置处理器，来对 Bean 可用后以及销毁前进行管理。和 XML 指定单个 Bean 的 init 和 destory 方法不同，后置处理器对于所有 IOC 的 Bean 实例逐一处理。我们需要实现 BeanPostProcessor 来提供后置处理器的实现，以管理 Bean 实例。在 Bean 被注入后，即交给后置处理器完成 init 方法，之后正常使用。在容器销毁前，交给后置处理器完成 destory 方法，之后进行 gc。

```java
public class Listener implements BeanPostProcessor {
    @Override
    public Object postProcessBeforeInitialization(Object o, String s) throws BeansException {
        System.out.println("init");
        return o;
    }

    @Override
    public Object postProcessAfterInitialization(Object o, String s) throws BeansException {
        System.out.println("destory");
        return o;
    }
}
```

注意，后置处理器必须在 XML 中进行初始化一个 Bean。

```xml
<bean class="com.mazhangjing.test.Listener" />
```